In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('/opt/ml/code/mmdetection_trash/configs/trash/swin_t/swin_htc.py')

PREFIX = '../../input/data/'

epoch = 12

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_data0.json'
#cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'valid_data0.json'
#cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
#cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = '/opt/ml/code/mmdetection_trash/work_dirs/swin_base_htc_0'

#cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
f_n = 'epoch_31'
checkpoint_path = os.path.join(cfg.work_dir, f'{f_n}.pth')

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

Use load_from_local loader
The model and loaded state dict do not match exactly

size mismatch for rpn_head.rpn_cls.weight: copying a param with shape torch.Size([3, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([7, 256, 1, 1]).
size mismatch for rpn_head.rpn_cls.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([7]).
size mismatch for rpn_head.rpn_reg.weight: copying a param with shape torch.Size([12, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([28, 256, 1, 1]).
size mismatch for rpn_head.rpn_reg.bias: copying a param with shape torch.Size([12]) from checkpoint, the shape in current model is torch.Size([28]).


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 2.8 task/s, elapsed: 304s, ETA:     0s

In [9]:
import json
import pandas as pd

images = []
anns = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()
class_num = 11

def make_img(imgph):
    answer=dict()
    answer['license']=0
    answer['url']=None
    answer['file_name']=imgph
    answer['height']=512
    answer['width']=512
    answer['date_captured']=None
    answer['id']=9999
    return answer

def make_ann(img,annin):
    answers = []
    for j in range(class_num):
        tempanns = []
        if len(annin[0][j]) > 0:
            for o in annin[0][j]:
                ann = dict()
                ann['id'] = 9999
                ann['image_id'] = img['id']
                ann['category_id'] = str(j)
                ann['segmentation'] = []
                ann['area'] = 0
                ann['bbox'] = list(o[0:4])
                ann['iscrowd'] = 0
                tempanns.append(ann)
        if len(annin[1][j]) > 0:
            for idx, o in enumerate(annin[1][j]):
                templist = []
                if len(o) != 512:
                    for oo in o:
                        mask = []
                        for xidx,a in enumerate(oo):
                            for yidx,b in enumerate(a):
                                if b==True:
                                    mask.append(xidx)
                                    mask.append(yidx)
                        templist.append(mask)
                else:
                    mask = []
                    for xidx,a in enumerate(o):
                        for yidx,b in enumerate(a):
                            if b==True:
                                mask.append(xidx)
                                mask.append(yidx)
                    templist.append(mask)
                tempanns[idx]['segmentation'] = templist
                tempanns[idx]['area'] = len(templist[0])
            answers.extend(tempanns)
    return answers

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [10]:
from tqdm.notebook import tqdm_notebook
for i, out in enumerate(output):
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    
    img = make_img(image_info['file_name'])
    
    img['id'] = i + 2617
    
    images.append(img)
    
    ann = make_ann(img,out)
    
    anns.extend(ann)
    if i % 50 == 0:
        print(len(anns), len(images))

0 1
141 51
274 101
592 151
1011 201
1470 251
1890 301
2135 351
2240 401
2456 451
2531 501
2884 551
3217 601
3729 651
4151 701
4445 751
4482 801


In [11]:
with open('../../input/data/train_data0.json') as f:
    info = json.load(f)

    for i in images:
        info['images'].append(i)
    for idx,an in enumerate(anns):
        an['id'] = idx + 21116
        info['annotations'].append(an)

    with open('./newtrain.json', 'w') as w:
        json.dump(info,w, ensure_ascii=False)
    print('done')

TypeError: Object of type float32 is not JSON serializable

In [15]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[0][j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

filename = 'swin_htc_00'
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{filename}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.014217303 155.39651 69.92909 257.15866 165...,batch_01_vt/0021.jpg
1,0 0.037008937 285.39294 343.8059 307.50024 377...,batch_01_vt/0028.jpg
2,1 0.9107458 61.774628 171.37961 368.94562 384....,batch_01_vt/0031.jpg
3,2 0.97098887 293.64658 158.49358 363.59125 228...,batch_01_vt/0032.jpg
4,0 0.16284999 38.874035 459.57465 93.51427 472....,batch_01_vt/0070.jpg
